# Reproduction of Adult dataset experiments

In this notebook we reproduce the results from Table 2 of the DECAF paper. We compare various methods for generating debiased data using the DECAF model against synthetic data generated using benchmark models GAN, WGAN-GP and FairGAN. As described in the paper we run all experiments (as implemented in this notebook) 10 times and avarage the results.

In [9]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

from data import load_adult, preprocess_adult
from metrics import DP, FTU
from train import train_decaf, train_fairgan, train_vanilla_gan, train_wgan_gp


## Loading data

In [10]:
dataset = load_adult()
dataset.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


Preprocess the data next in order to make it suitable for training models on.

In [11]:
dataset = preprocess_adult(dataset)
dataset.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.301370,0.833333,0.043338,0.000000,0.800000,0.333333,0.615385,0.6,0.0,1.0,0.02174,0.0,0.397959,0.0,1.0
1,0.452055,0.166667,0.047277,0.000000,0.800000,0.000000,0.307692,0.4,0.0,1.0,0.00000,0.0,0.122449,0.0,1.0
2,0.287671,0.000000,0.137244,0.200000,0.533333,0.166667,0.461538,0.6,0.0,1.0,0.00000,0.0,0.397959,0.0,1.0
3,0.493151,0.000000,0.150212,0.133333,0.400000,0.000000,0.461538,0.4,1.0,1.0,0.00000,0.0,0.397959,0.0,1.0
4,0.150685,0.000000,0.220703,0.000000,0.800000,0.000000,0.384615,0.0,1.0,0.0,0.00000,0.0,0.397959,0.3,1.0


Split the dataset into train and test folds. Test fold size is 2000.

In [12]:
# Split data into train and testing sets
dataset_train, dataset_test = train_test_split(dataset, test_size=2000,
                                               stratify=dataset['income'])

print('Size of train set:', len(dataset_train))
print('Size of test set:', len(dataset_test))

Size of train set: 28162
Size of test set: 2000


### Defining the DAG

We need to define a DAG which captures the biases of the dataset. As described in the DECAF paper normally a causal discovery algorithm is used. In this notebook we simply copy the DAG which as described in the Zhang et al. paper which is the one also used in the DECAF paper.

In [13]:
# Define DAG for Adult dataset
dag = [
    # Edges from race
    ['race', 'occupation'],
    ['race', 'income'],
    ['race', 'hours-per-week'],
    ['race', 'education'],
    ['race', 'marital-status'],

    # Edges from age
    ['age', 'occupation'],
    ['age', 'hours-per-week'],
    ['age', 'income'],
    ['age', 'workclass'],
    ['age', 'marital-status'],
    ['age', 'education'],
    ['age', 'relationship'],
    
    # Edges from sex
    ['sex', 'occupation'],
    ['sex', 'marital-status'],
    ['sex', 'income'],
    ['sex', 'workclass'],
    ['sex', 'education'],
    ['sex', 'relationship'],
    
    # Edges from native country
    ['native-country', 'marital-status'],
    ['native-country', 'hours-per-week'],
    ['native-country', 'education'],
    ['native-country', 'workclass'],
    ['native-country', 'income'],
    ['native-country', 'relationship'],
    
    # Edges from marital status
    ['marital-status', 'occupation'],
    ['marital-status', 'hours-per-week'],
    ['marital-status', 'income'],
    ['marital-status', 'workclass'],
    ['marital-status', 'relationship'],
    ['marital-status', 'education'],
    
    # Edges from education
    ['education', 'occupation'],
    ['education', 'hours-per-week'],
    ['education', 'income'],
    ['education', 'workclass'],
    ['education', 'relationship'],
    
    # All remaining edges
    ['occupation', 'income'],
    ['hours-per-week', 'income'],
    ['workclass', 'income'],
    ['relationship', 'income'],
]

def dag_to_idx(df, dag):
    """Convert columns in a DAG to the corresponding indices."""

    dag_idx = []
    for edge in dag:
        dag_idx.append([df.columns.get_loc(edge[0]), df.columns.get_loc(edge[1])])

    return dag_idx

# Convert the DAG to one that can be provided to the DECAF model
dag_seed = dag_to_idx(dataset, dag)
print(dag_seed)

[[8, 6], [8, 14], [8, 12], [8, 3], [8, 5], [0, 6], [0, 12], [0, 14], [0, 1], [0, 5], [0, 3], [0, 7], [9, 6], [9, 5], [9, 14], [9, 1], [9, 3], [9, 7], [13, 5], [13, 12], [13, 3], [13, 1], [13, 14], [13, 7], [5, 6], [5, 12], [5, 14], [5, 1], [5, 7], [5, 3], [3, 6], [3, 12], [3, 14], [3, 1], [3, 7], [6, 14], [12, 14], [1, 14], [7, 14]]


It's also necessary to define edges we want to remove from the DAG in order to meet the various fairness criteria described in the paper.

In [14]:
def create_bias_dict(df, edge_map):
    """
    Convert the given edge tuples to a bias dict used for generating
    debiased synthetic data.
    """
    bias_dict = {}
    for key, val in edge_map.items():
        bias_dict[df.columns.get_loc(key)] = [df.columns.get_loc(f) for f in val]
    
    return bias_dict

# Bias dictionary to satisfy FTU
bias_dict_ftu = create_bias_dict(dataset, {'income': ['sex']})
print('Bias dict FTU:', bias_dict_ftu)

# Bias dictionary to satisfy DP
bias_dict_dp = create_bias_dict(dataset, {'income': [
    'occupation', 'hours-per-week', 'marital-status', 'education', 'sex',
    'workclass', 'relationship']})
print('Bias dict DP:', bias_dict_dp)

# Bias dictionary to satisfy CF
bias_dict_cf = create_bias_dict(dataset, {'income': [
    'marital-status', 'sex']})
print('Bias dict CF:', bias_dict_cf)

Bias dict FTU: {14: [9]}
Bias dict DP: {14: [6, 12, 5, 3, 9, 1, 7]}
Bias dict CF: {14: [5, 9]}


## Experiments

We have loaded and preprocessed the data and we are ready to run the experiments. For each experiment we train a generative model, sample synthetic data from the trained model and then obtain metrics by training and evaluating a downstream multi-layer perceptron using the test fold we generated in the previous section. We use the MLP model from `sklearn` with default parameters which matches the settings described in Appendix D of the paper.

In [15]:
X_train, y_train = dataset_train.drop(columns=['income']), dataset_train['income']
base_clf = MLPClassifier()
base_clf.fit(X_train, y_train)

def eval_model(dataset_train, dataset_test, use_base_clf=True):
    """Helper function that prints evaluation metrics."""

    X_train, y_train = dataset_train.drop(columns=['income']), dataset_train['income']
    X_test, y_test = dataset_test.drop(columns=['income']), dataset_test['income']

    if use_base_clf:
        y_train = base_clf.predict(X_train)

    clf = MLPClassifier()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    auroc = roc_auc_score(y_test, y_pred)
    dp = DP(clf, X_test)
    ftu = FTU(clf, X_test)

    return {'precision': precision, 'recall': recall, 'auroc': auroc,
            'dp': dp, 'ftu': ftu}

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


### Original dataset

As a benchmark we want to first train the downstream model on the original dataset.

In [16]:
eval_model(dataset_train, dataset_test, use_base_clf=False)

{'precision': 0.8794788273615635,
 'recall': 0.8988015978695073,
 'auroc': 0.7636578270472035,
 'dp': 0.21241924075308738,
 'ftu': 0.06600000000000006}

In the following sections we train various models in order to reproduce the results from Table 2 of the DECAF paper.

### GAN

In [17]:
synth_data = train_vanilla_gan(dataset_train)
synth_data.head()

2022-01-26 15:13:47.684628: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-26 15:13:47.730040: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Synthetic data generation: 100%|██████████| 221/221 [00:02<00:00, 100.82it/s]


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.066293,0.166667,0.038547,0.133333,0.066667,0.166667,0.615385,0.4,0.0,1.0,-0.032644,-0.004545,0.072970,0.350,0.0
1,0.074198,0.500000,0.041145,0.266667,0.866667,0.000000,1.000000,0.2,0.5,1.0,-0.035194,-0.003407,0.075435,0.275,1.0
2,0.068533,0.000000,0.040603,0.733333,0.000000,0.500000,0.307692,0.6,0.0,0.0,-0.030734,-0.010964,0.075320,0.425,1.0
3,0.081151,0.000000,0.036435,0.866667,0.133333,0.166667,0.692308,0.4,1.0,0.0,-0.041740,-0.005363,0.074609,0.250,1.0
4,0.072031,0.166667,0.037350,0.866667,0.666667,0.500000,0.230769,0.6,0.5,0.0,-0.033487,-0.007045,0.070043,0.750,0.0


In [18]:
eval_model(synth_data, dataset_test)

{'precision': 0.751, 'recall': 1.0, 'auroc': 0.5, 'dp': 0.0, 'ftu': 0.0}

### WGAN-GP

In [19]:
synth_data = train_wgan_gp(dataset_train)
synth_data.head()

Synthetic data generation: 100%|██████████| 57/57 [00:00<00:00, 71.42it/s]


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,-0.484268,0.333333,-0.282228,0.666667,0.200000,0.833333,0.307692,0.0,1.00,0.0,0.013445,0.231709,0.741034,0.800,0.0
1,-0.537463,1.000000,-0.239931,0.600000,0.666667,0.833333,0.769231,0.0,0.75,0.0,-0.048446,0.174137,0.783880,0.975,1.0
2,-0.403302,0.833333,-0.242282,0.333333,0.533333,0.000000,1.000000,0.0,0.75,1.0,0.016809,0.212029,0.706402,0.625,1.0
3,-0.508704,0.666667,-0.189006,1.000000,0.333333,1.000000,0.846154,1.0,1.00,0.0,0.087113,0.185299,0.778692,0.875,1.0
4,-0.495381,0.500000,-0.227208,0.066667,0.666667,0.000000,0.615385,0.8,0.00,1.0,-0.024477,0.203735,0.768136,1.000,1.0


In [20]:
eval_model(synth_data, dataset_test)

{'precision': 0.751, 'recall': 1.0, 'auroc': 0.5, 'dp': 0.0, 'ftu': 0.0}

### FairGAN

In [21]:
synth_data = train_fairgan(dataset_train)
synth_data.head()

INFO:tensorflow:Restoring parameters from fair-9
burning in


2022-01-26 15:13:55.524047: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:196] None of the MLIR optimization passes are enabled (registered 0 passes)


generating


/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.895464,0.000000,0.308918,0.067286,0.000000,0.131079,0.000000,0.278640,1.752207,0.0,0.0,0.423585,0.0,0.471339,1.0
1,0.000000,0.104030,0.000000,0.645687,0.454629,0.018567,0.588073,0.000000,0.085750,0.0,0.0,0.491787,0.0,0.623588,1.0
2,0.027066,0.213806,0.497826,0.557692,0.587919,0.596714,0.858378,0.000000,0.621208,0.0,0.0,0.383516,0.0,1.051334,1.0
3,0.000000,0.000000,0.000000,0.383852,0.000000,0.092355,0.440810,0.000000,0.943324,0.0,0.0,0.595602,0.0,0.000000,1.0
4,0.000000,0.091245,0.480436,0.466834,0.087119,0.179120,0.347991,0.154661,0.236591,0.0,0.0,0.591424,0.0,1.015640,1.0


In [22]:
eval_model(synth_data, dataset_test, use_base_clf=False)

{'precision': 0.8200122025625382,
 'recall': 0.8948069241011984,
 'auroc': 0.6512187230947759,
 'dp': 0.07822718293922482,
 'ftu': 0.0050000000000000044}

### DECAF

#### DECAF-ND

In [23]:
synth_data = train_decaf(dataset_train, dag_seed)
synth_data.head()

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
27139,0.436500,4.319716e-03,0.241909,0.245294,0.641810,0.175803,0.233618,0.779197,6.271827e-14,0.001254,0.002225,0.071768,0.545782,3.127366e-01,0.0
16615,0.049367,2.083480e-07,0.070874,0.179708,0.633827,0.393200,0.440628,0.836775,8.607731e-07,0.001417,0.002778,0.001703,0.179432,5.485995e-04,1.0
16341,0.240197,9.649503e-06,0.219023,0.133397,0.514405,0.405465,0.276106,0.135312,6.238532e-09,0.001455,0.033405,0.000163,0.368845,6.467823e-09,0.0
18333,0.141543,4.923712e-04,0.045966,0.085631,0.735626,0.047681,0.104996,0.465879,2.409033e-10,1.000000,0.001248,0.000750,0.550966,1.520371e-06,0.0
12348,0.525342,3.014280e-07,0.164513,0.080996,0.657318,0.011562,0.223437,0.623650,8.150314e-12,0.001593,0.001170,0.000605,0.303026,5.330425e-02,0.0


In [24]:
eval_model(synth_data, dataset_test)

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


{'precision': 0.8641744548286604,
 'recall': 0.9234354194407457,
 'auroc': 0.7428422077123407,
 'dp': 0.1758568912865186,
 'ftu': 0.03699999999999992}

#### DECAF-FTU

In [25]:
synth_data = train_decaf(dataset_train, dag_seed, biased_edges=bias_dict_ftu)
synth_data.head()

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
27139,0.509642,8.872179e-01,0.224771,0.197451,0.685305,0.009582,0.083535,0.276030,5.131579e-09,0.996559,0.018437,0.000469,0.558874,0.000331,0.0
16615,0.184314,3.348884e-05,0.222814,0.396268,0.745416,0.157064,0.122975,0.137961,2.393370e-03,0.850840,0.012695,0.003973,0.456888,0.446396,1.0
16341,0.077989,2.970788e-03,0.069831,0.157137,0.748655,0.008631,0.153496,0.432476,4.253140e-01,1.000000,0.035055,0.080201,0.545748,0.009890,0.0
18333,0.344249,1.017547e-08,0.198463,0.196320,0.641043,0.053237,0.303360,0.424696,9.073946e-13,1.000000,0.000826,0.009673,0.301549,0.000001,0.0
12348,0.435792,2.178438e-07,0.085216,0.468681,0.789598,0.223773,0.067899,0.246224,6.250097e-04,0.999756,0.001243,0.000048,0.375688,0.674506,0.0


In [26]:
eval_model(synth_data, dataset_test)

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


{'precision': 0.8582871226124461,
 'recall': 0.9274300932090546,
 'auroc': 0.7327913518254109,
 'dp': 0.18270006351312174,
 'ftu': 0.03049999999999997}

#### DECAF-CF

In [27]:
synth_data = train_decaf(dataset_train, dag_seed, biased_edges=bias_dict_cf)
synth_data.head()

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
27139,0.142002,1.060784e-04,0.239512,0.052298,0.744922,0.036939,0.085835,0.265633,9.939086e-01,1.000000,0.003970,0.004088,0.558089,2.005967e-04,1.0
16615,0.068295,9.724969e-05,0.074014,0.137814,0.825064,0.336780,0.183999,0.419002,1.008208e-16,1.000000,0.004765,0.059977,0.253592,4.350131e-10,1.0
16341,0.514636,8.853149e-01,0.112634,0.040463,0.822149,0.454076,0.749997,0.624520,5.806358e-14,1.000000,0.007169,0.012816,0.409016,4.402062e-04,0.0
18333,0.529593,1.739735e-07,0.171975,0.068834,0.594917,0.178869,0.370754,0.748682,2.103174e-03,1.000000,0.001674,0.001170,0.299214,6.460656e-07,0.0
12348,0.172862,6.052432e-02,0.033594,0.222110,0.491555,0.002657,0.297504,0.361277,7.329545e-10,0.880917,0.002926,0.022809,0.281122,3.852306e-05,0.0


In [28]:
eval_model(synth_data, dataset_test)

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


{'precision': 0.8565297363580625,
 'recall': 0.9300932090545939,
 'auroc': 0.7301068454911522,
 'dp': 0.17671651826256074,
 'ftu': 0.019499999999999962}

#### DECAF-DP

In [29]:
synth_data = train_decaf(dataset_train, dag_seed, biased_edges=bias_dict_dp)
synth_data.head()

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
27139,0.106765,1.809077e-02,0.055046,0.309620,0.761848,0.019992,0.354194,0.087959,5.836950e-10,1.000000,0.020551,0.021853,0.256658,3.356911e-08,0.0
16615,0.181248,5.591948e-04,0.122723,0.368025,0.491348,0.342309,0.477082,0.351601,1.774761e-15,0.985308,0.009026,0.000756,0.548007,8.095511e-09,1.0
16341,0.532931,9.463285e-07,0.173834,0.072664,0.677506,0.006025,0.239754,0.488020,2.668208e-15,1.000000,0.025808,0.000107,0.558242,1.230794e-09,0.0
18333,0.050200,5.272870e-05,0.059588,0.220444,0.684066,0.467464,0.275988,0.223379,3.235187e-03,1.000000,0.001697,0.020862,0.185860,1.680196e-09,0.0
12348,0.533758,1.163491e-06,0.088655,0.067822,0.706045,0.009546,0.335263,0.455945,3.332579e-08,1.000000,0.005363,0.010634,0.553603,1.261160e-06,1.0


In [30]:
eval_model(synth_data, dataset_test)

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


{'precision': 0.8619402985074627,
 'recall': 0.9227696404793608,
 'auroc': 0.7384932539746202,
 'dp': 0.16458810668850699,
 'ftu': 0.023499999999999965}